In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

# Custom CartPole Environment
class CustomCartPole:
    def __init__(self):
        # Constants
        self.gravity = 9.8
        self.mass_cart = 1.0
        self.mass_pole = 0.1
        self.total_mass = self.mass_cart + self.mass_pole
        self.length = 0.5  # Half the length of the pole
        self.pole_mass_length = self.mass_pole * self.length
        self.force_mag = 10.0
        self.tau = 0.02  # Time step (20 ms)
        self.theta_threshold_radians = 12 * 2 * np.pi / 360  # 12 degrees
        self.x_threshold = 2.4  # Cart position threshold (meters)

        # State variables
        self.state = None
        self.steps_beyond_done = None

    def reset(self):
        # Reset the state to a random small initial value
        self.state = np.random.uniform(low=-0.05, high=0.05, size=(4,))
        self.steps_beyond_done = None
        return np.array(self.state, dtype=np.float32)

    def step(self, action):
        # Get the current state
        x, x_dot, theta, theta_dot = self.state

        # Force applied based on action (0: left, 1: right)
        force = self.force_mag if action == 1 else -self.force_mag

        # Dynamics equations
        costheta = np.cos(theta)
        sintheta = np.sin(theta)
        temp = (force + self.pole_mass_length * theta_dot**2 * sintheta) / self.total_mass
        theta_acc = (self.gravity * sintheta - costheta * temp) / \
                    (self.length * (4.0/3.0 - self.mass_pole * costheta**2 / self.total_mass))
        x_acc = temp - self.pole_mass_length * theta_acc * costheta / self.total_mass

        # Update the state using Euler's method
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * x_acc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * theta_acc
        self.state = (x, x_dot, theta, theta_dot)

        # Check if the episode is done
        done = bool(
            x < -self.x_threshold
            or x > self.x_threshold
            or theta < -self.theta_threshold_radians
            or theta > self.theta_threshold_radians
        )

        # Compute the reward
        if not done:
            reward = 1.0
        elif self.steps_beyond_done is None:
            self.steps_beyond_done = 0
            reward = 1.0
        else:
            self.steps_beyond_done += 1
            reward = 0.0

        return np.array(self.state, dtype=np.float32), reward, done, {}

    def render(self):
        x, _, theta, _ = self.state
        print(f"Cart Position: {x:.2f}, Pole Angle: {theta:.2f}")

    def close(self):
        pass


# Q-Network for DQN
class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)


# DQN Agent
class DQNAgent:
    def __init__(self, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.q_network = QNetwork(state_dim, action_dim)
        self.target_network = QNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=1e-3)
        self.replay_buffer = []
        self.buffer_capacity = 10000
        self.batch_size = 64
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

    def act(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            return torch.argmax(self.q_network(state_tensor)).item()

    def store_transition(self, transition):
        if len(self.replay_buffer) >= self.buffer_capacity:
            self.replay_buffer.pop(0)
        self.replay_buffer.append(transition)

    def sample_batch(self):
        indices = np.random.choice(len(self.replay_buffer), self.batch_size)
        batch = [self.replay_buffer[idx] for idx in indices]
        return batch

    def learn(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        batch = self.sample_batch()
        states, actions, rewards, next_states, dones = zip(*batch)
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)

        q_values = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze()
        with torch.no_grad():
            max_next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + self.gamma * max_next_q_values * (1 - dones)

        loss = nn.MSELoss()(q_values, target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())


# Training Loop
env = CustomCartPole()
state_dim = 4
action_dim = 2
agent = DQNAgent(state_dim, action_dim)

num_episodes = 500
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    for t in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        agent.store_transition((state, action, reward, next_state, done))
        state = next_state
        total_reward += reward

        agent.learn()

        if done:
            break

    agent.update_target_network()
    agent.epsilon = max(agent.epsilon_min, agent.epsilon * agent.epsilon_decay)
    print(f"Episode {episode}, Total Reward: {total_reward}")


Episode 0, Total Reward: 16.0
Episode 1, Total Reward: 18.0
Episode 2, Total Reward: 24.0


C:\Users\Pratik\AppData\Local\Temp\ipykernel_19536\1343452155.py:135: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  states = torch.FloatTensor(states)


Episode 3, Total Reward: 46.0
Episode 4, Total Reward: 12.0
Episode 5, Total Reward: 18.0
Episode 6, Total Reward: 23.0
Episode 7, Total Reward: 18.0
Episode 8, Total Reward: 21.0
Episode 9, Total Reward: 61.0
Episode 10, Total Reward: 13.0
Episode 11, Total Reward: 25.0
Episode 12, Total Reward: 18.0
Episode 13, Total Reward: 16.0
Episode 14, Total Reward: 18.0
Episode 15, Total Reward: 34.0
Episode 16, Total Reward: 30.0
Episode 17, Total Reward: 27.0
Episode 18, Total Reward: 21.0
Episode 19, Total Reward: 22.0
Episode 20, Total Reward: 13.0
Episode 21, Total Reward: 18.0
Episode 22, Total Reward: 47.0
Episode 23, Total Reward: 25.0
Episode 24, Total Reward: 19.0
Episode 25, Total Reward: 31.0
Episode 26, Total Reward: 25.0
Episode 27, Total Reward: 26.0
Episode 28, Total Reward: 14.0
Episode 29, Total Reward: 37.0
Episode 30, Total Reward: 27.0
Episode 31, Total Reward: 37.0
Episode 32, Total Reward: 38.0
Episode 33, Total Reward: 32.0
Episode 34, Total Reward: 60.0
Episode 35, Tot